In [2]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms

Hyper Parameters

In [3]:
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

MNIST Dataset

In [4]:
train_dataset = dsets.MNIST(
    root='../../data', train=True, transform=transforms.ToTensor(), download=True)

test_dataset = dsets.MNIST(
    root='../../data', train=False, transform=transforms.ToTensor())

Data Loader (Input Pipeline)

In [5]:
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset, batch_size=batch_size, shuffle=False)

Neural Network Model (1 hidden layer)

In [6]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [7]:
net = Net(input_size, hidden_size, num_classes)

Loss and Optimizer

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

Train the Model

In [9]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Convert torch tensor to Variable
        images = images.view(-1, 28 * 28)

        # Forward + Backward + Optimize
        optimizer.zero_grad()  # zero the gradient buffer
        outputs = net(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f' %
                  (epoch + 1, num_epochs, i + 1,
                   len(train_dataset) // batch_size, loss.item()))

Epoch [1/5], Step [100/600], Loss: 0.4392
Epoch [1/5], Step [200/600], Loss: 0.2366
Epoch [1/5], Step [300/600], Loss: 0.2125
Epoch [1/5], Step [400/600], Loss: 0.2087
Epoch [1/5], Step [500/600], Loss: 0.2037
Epoch [1/5], Step [600/600], Loss: 0.0945
Epoch [2/5], Step [100/600], Loss: 0.1767
Epoch [2/5], Step [200/600], Loss: 0.1293
Epoch [2/5], Step [300/600], Loss: 0.1616
Epoch [2/5], Step [400/600], Loss: 0.0632
Epoch [2/5], Step [500/600], Loss: 0.0570
Epoch [2/5], Step [600/600], Loss: 0.2703
Epoch [3/5], Step [100/600], Loss: 0.1002
Epoch [3/5], Step [200/600], Loss: 0.0762
Epoch [3/5], Step [300/600], Loss: 0.0502
Epoch [3/5], Step [400/600], Loss: 0.0854
Epoch [3/5], Step [500/600], Loss: 0.0161
Epoch [3/5], Step [600/600], Loss: 0.0661
Epoch [4/5], Step [100/600], Loss: 0.0297
Epoch [4/5], Step [200/600], Loss: 0.0376
Epoch [4/5], Step [300/600], Loss: 0.1031
Epoch [4/5], Step [400/600], Loss: 0.0485
Epoch [4/5], Step [500/600], Loss: 0.0706
Epoch [4/5], Step [600/600], Loss:

Test the Model

In [10]:
correct = 0
total = 0
for images, labels in test_loader:
    images = images.view(-1, 28 * 28)
    outputs = net(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

print('Accuracy of the network on the 10000 test images: %d %%' %
      (100 * correct / total))

Accuracy of the network on the 10000 test images: 98 %


Save the Model

In [13]:
torch.save(net.state_dict(), 'model.pkl')